In [17]:
!pip install pysqlite3
!pip install pysqlite3-binary
!pip install pypdf

In [18]:
!pip install -U langchain-community

In [19]:
!pip install faiss-cpu

In [20]:
from langchain.document_loaders import TextLoader, PyPDFLoader # load text from pdf and load text files
from langchain.text_splitter import RecursiveCharacterTextSplitter # raw data will be splited into multiple chunks
from langchain.embeddings import HuggingFaceBgeEmbeddings, OpenAIEmbeddings #, GooglePalmEmbeddings # we need embedding model for converting text to numbers. So we will use a model from HuggingFace which is free of cost or we can also use OpenAIEmbeddings which is paid.
from langchain.vectorstores import FAISS
import faiss
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

In [21]:
# loads the text content from the txt file
text_loader = TextLoader("state_of_the_union.txt")
text_document = text_loader.load()
print(text_document[0].page_content[:100]) # prints first 100 character of the text document

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and th


In [22]:
from langchain.document_loaders import PyPDFLoader
file_path = "resume1.pdf"
pdf_loader = PyPDFLoader(file_path)
pdf_pages = pdf_loader.load_and_split()
print(pdf_pages[0].page_content[:100])

Alice Johnson
Software Engineer
Summary
Enthusiastic software engineer with 5 years of experience in


In [23]:
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64) # chunk_size means each chunk will have 1024 chaaracters and chunk_overlap means 64 characters could be overlapped for relevance for mantaining context
split_texts = doc_splitter.split_documents(pdf_pages)
print(len(split_texts))

1


In [24]:
# embedding text to numbers
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
hf_embed = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)
text = split_texts[0].page_content
hf_embed_result = hf_embed.embed_documents([text])
# print(len(hf_embed_result))
print(len(hf_embed_result[0]))

/tmp/ipython-input-3812657093.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embed = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

768


In [25]:
# store embeddings into vector store
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
hf_embed = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)

faiss_index = FAISS.from_documents(split_texts, hf_embed)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
# perform similarity search and print the top two most similar documents
search_result = faiss_index.similarity_search_with_score("What is the candidates skill set", k=2)
print(search_result)

[(Document(id='c7ba8dd0-7344-4123-a436-abf3e7d4a3de', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-20T21:22:55+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-20T21:22:55+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resume1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Alice Johnson\nSoftware Engineer\nSummary\nEnthusiastic software engineer with 5 years of experience in developing scalable web applications.\nExperience\nSoftware Engineer - TechCorp (2020-2025)\nWorked on full-stack applications using React and Node.js.\nJunior Developer - WebStart (2018-2020)\nAssisted in backend development and database management.\nSkills\nPython, JavaScript, React, Node.js, SQL\nEducation\nB.Sc. Computer Science - State University (2018)'), np.float32(1.4143939))]


In [27]:
# downlaod result and save on loacl
faiss_index.save_local("faiss_index")

In [28]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
hf_embed = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)

# load the persisted FAISS index(must match embedding model used during creation)
faiss_index = FAISS.load_local("faiss_index", hf_embed, allow_dangerous_deserialization=True)

# perfoem a similarity search with the loaded FAISS index
vector_search_result = faiss_index.similarity_search_with_score("What is the candidates skill set", k=2)
print(vector_search_result)

[(Document(id='c7ba8dd0-7344-4123-a436-abf3e7d4a3de', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-20T21:22:55+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-20T21:22:55+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resume1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Alice Johnson\nSoftware Engineer\nSummary\nEnthusiastic software engineer with 5 years of experience in developing scalable web applications.\nExperience\nSoftware Engineer - TechCorp (2020-2025)\nWorked on full-stack applications using React and Node.js.\nJunior Developer - WebStart (2018-2020)\nAssisted in backend development and database management.\nSkills\nPython, JavaScript, React, Node.js, SQL\nEducation\nB.Sc. Computer Science - State University (2018)'), np.float32(1.4143939))]
